In [ ]:
# 開発過程の補足

# この Jupyter Notebook ファイルは、開発過程を示すために作成されています。  
# セル単位でコードを実行しながら処理内容を確認する「検証用環境」として使用しています。  
# 実際の本番環境では VSCode 上でモジュールを作成し、AWS Lambda および Step Functions によりデータ処理を実行しています。

In [ ]:
# 開発履歴確認用

# セル1: 必要なライブラリのインポート
import os
import yaml
import json
import requests
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime
import logging
from pathlib import Path
import time

# 商品検索API：https://developer.yahoo.co.jp/webapi/shopping/v3/itemsearch.html

In [ ]:
# セル2: YahooAPIクラスの実装
class YahooShoppingAPI:
    """
    ヤフーショッピングAPIを使用して商品情報を取得するクラス
    
    このクラスはYahoo Shopping APIを使って、指定したストアIDやその他の条件で
    商品情報を検索・取得します。
    """
    
    def __init__(self, config_path=None):
        """
        YahooShoppingAPIクラスの初期化
        
        Parameters:
            config_path (str, optional): 設定ファイルのパス
        """
        # プロジェクトルートディレクトリの検出
        self.root_dir = self._find_project_root()
        
        # 環境変数の読み込み
        load_dotenv(os.path.join(self.root_dir, '.env'))
        
        # ディレクトリパスの設定
        self.data_dir = os.path.join(self.root_dir, 'data')
        self.log_dir = os.path.join(self.root_dir, 'logs')
        
        # ディレクトリが存在しない場合は作成
        os.makedirs(self.data_dir, exist_ok=True)
        os.makedirs(self.log_dir, exist_ok=True)
        
        # 設定ファイルの読み込み
        self.config = self._load_config(config_path)
        
        # APIキーの設定（環境変数から取得）
        self.client_id = os.getenv('YAHOO_CLIENT_ID')
        if not self.client_id:
            raise ValueError("YAHOO_CLIENT_IDが環境変数に設定されていません")
        
        # ログ設定
        self.setup_logging()
        
        # APIのベースURL
        self.base_url = "https://shopping.yahooapis.jp/ShoppingWebService/V3/itemSearch"
    
    def _find_project_root(self):
        """プロジェクトのルートディレクトリを検出する"""
        # 現在のファイルの絶対パスを取得
        current_dir = os.path.abspath(os.getcwd())
        
        # 親ディレクトリを探索
        path = Path(current_dir)
        while True:
            # .gitディレクトリがあればそれをルートとみなす
            if (path / '.git').exists():
                return str(path)
            
            # プロジェクトのルートを示す他のファイル/ディレクトリの存在チェック
            if (path / 'setup.py').exists() or (path / 'README.md').exists():
                return str(path)
            
            # これ以上上の階層がない場合は現在のディレクトリを返す
            if path.parent == path:
                return str(path)
            
            # 親ディレクトリへ
            path = path.parent
    
    def _load_config(self, config_path=None):
        """設定ファイルを読み込む"""
        # デフォルトのパス
        if config_path is None:
            config_path = os.path.join(self.root_dir, 'config', 'settings.yaml')
        
        print(f"設定ファイルパス: {config_path}")
        
        try:
            if not os.path.exists(config_path):
                raise FileNotFoundError(f"設定ファイルが見つかりません: {config_path}")
                
            # YAMLファイルを読み込む
            with open(config_path, 'r', encoding='utf-8') as f:
                config = yaml.safe_load(f)
            
            # ヤフーショッピングの設定確認
            if 'yahoo_shopping' not in config:
                config['yahoo_shopping'] = {}
            
            # デフォルトの出力設定（なければ設定）
            if 'output' not in config['yahoo_shopping']:
                config['yahoo_shopping']['output'] = {
                    'csv_filename': 'yahoo_shopping_items.csv'
                }
            
            # ストアIDリストの確認
            if 'store_ids' not in config['yahoo_shopping']:
                config['yahoo_shopping']['store_ids'] = []
                print("警告: ストアIDが設定されていません")
            
            print("設定ファイルを正常に読み込みました")
            return config
        except Exception as e:
            print(f"設定ファイルの読み込みエラー: {str(e)}")
            # エラーを上位に伝播させる
            raise
    
    def setup_logging(self):
        """ログ機能のセットアップ"""
        # すでに存在するハンドラを削除（重複を防ぐため）
        for handler in logging.root.handlers[:]:
            logging.root.removeHandler(handler)
        
        # ログファイルパスの設定
        log_filename = f'yahoo_api_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'
        log_file = os.path.join(self.log_dir, log_filename)
        
        # 基本設定
        logging.basicConfig(
            filename=log_file,
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            encoding='utf-8'
        )
        
        # コンソールにもログを出力
        console = logging.StreamHandler()
        console.setLevel(logging.INFO)
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        console.setFormatter(formatter)
        logging.getLogger('').addHandler(console)
        
        # ログファイルの場所を明示的に表示
        print(f"ログファイル出力先: {log_file}")
        logging.info(f"ログ機能の初期化が完了しました: {log_file}")
    
    def search_items(self, store_id, query="", page=1, sort="+price", max_results=100, price_from=None, price_to=10000):
        """
        指定したストアIDで商品を検索する
        
        Parameters:
            store_id (str): ストアID
            query (str, optional): 検索キーワード
            page (int, optional): ページ番号
            sort (str, optional): ソート条件
            max_results (int, optional): 1ページあたりの最大取得件数
            price_from (int, optional): 最低価格
            price_to (int, optional): 最高価格
                
        Returns:
            dict: APIレスポンス
        """
        # 開始位置の計算（1ページ目=1, 2ページ目=101, 3ページ目=201, ...)
        start_position = (page - 1) * 100 + 1
        
        # 最後のページ（10ページ目、start=901）の場合は特別処理
        if start_position == 901:
            results = 99  # 10ページ目は99件だけ取得（901～999まで）
        else:
            # 1000件制限に対応するための調整
            results = min(max_results, 1000 - start_position + 1)
        
        # 取得件数が0以下になる場合は、APIの制限を超えているためエラー
        if results <= 0:
            logging.error(f"APIの制限（1000件）を超えるリクエストです: start={start_position}")
            return {"error": "APIの制限（1000件）を超えるリクエスト", "hits": []}
        
        # パラメータの設定
        params = {
            'appid': self.client_id,
            'seller_id': store_id,
            'availability': 1,       # 在庫あり
            'condition': 'new',      # 新品
            'shipping_area': 23,     # 愛知県
            'sort': sort,            # 価格の安い順
            'start': start_position, # 開始位置
            'results': results,      # 調整後の取得件数
            'shipping_lead_time': 4  # 4日以内に発送
        }
        
        # 価格範囲の設定
        if price_from is not None:
            params['price_from'] = price_from
        if price_to is not None:
            params['price_to'] = price_to
        
        # 検索キーワードがある場合は追加
        if query:
            params['query'] = query
        
        try:
            # APIリクエスト
            logging.info(f"ストアID {store_id} の商品検索を開始 (開始位置: {start_position}, 取得件数: {results})")
            response = requests.get(self.base_url, params=params)
            
            # HTTPステータスコードのチェック
            response.raise_for_status()
            
            # JSONに変換
            data = response.json()
            
            logging.info(f"ストアID {store_id} の商品検索結果: {data.get('totalResultsReturned', 0)}件")
            return data
        except requests.exceptions.RequestException as e:
            logging.error(f"APIリクエストエラー: {str(e)}")
            return {"error": str(e), "hits": []}
            
    
    def extract_product_data(self, hit):
        """
        APIレスポンスから必要な商品情報を抽出する
        
        Parameters:
            hit (dict): ヒット情報
            
        Returns:
            dict: 抽出した商品情報
        """
        # JANコードの抽出（空の場合もある）
        jan_code = hit.get('janCode', '')
        
        # 基本情報
        name = hit.get('name', '')
        
        # 価格情報（税込）- 税抜価格が直接提供されていないようなので税込価格を使用
        price = hit.get('price', 0)
        
        # 送料情報
        shipping = hit.get('shipping', {})
        shipping_name = shipping.get('name', '') if shipping else ''
        
        # ポイント情報（ストアポイント）
        point = hit.get('point', {})
        ly_limited_bonus_amount = point.get('lyLimitedBonusAmount', 0) if point else 0
        
        # 発送予定日の取得
        delivery = hit.get('delivery', {})
        delivery_day = delivery.get('day', '') if delivery else ''
        
        # URL情報
        product_url = hit.get('url', '')
        
        # ストアID
        seller = hit.get('seller', {})
        store_id = seller.get('sellerId', '') if seller else ''
        
        # 必要な情報を辞書として返す
        return {
            'JAN': jan_code,
            '商品名': name,
            '価格': price,  # 税込価格
            '送料条件': shipping_name,
            'ストアポイント': ly_limited_bonus_amount,
            '発送予定日': delivery_day,
            '商品URL': product_url,
            'ストアID': store_id
        }
    
    def search_all_stores(self, query="", max_pages=1, save_to_csv=True):
        """
        すべてのストアIDに対して商品検索を実行する
        
        Parameters:
            query (str, optional): 検索キーワード
            max_pages (int, optional): 各ストアの最大ページ数
            save_to_csv (bool, optional): CSVに保存するかどうか
            
        Returns:
            tuple: (商品情報のデータフレーム, 検索統計情報のデータフレーム)
        """
        # 設定からストアIDリストを取得
        store_ids = self.config['yahoo_shopping']['store_ids']
        
        if not store_ids:
            logging.warning("ストアIDが設定されていません")
            return pd.DataFrame(), pd.DataFrame()
        
        # 全商品データの格納用リスト
        all_products = []
        
        # 検索統計情報の格納用リスト
        search_stats = []
        
        # 各ストアIDで検索
        for store_id in store_ids:
            for page in range(1, max_pages + 1):
                # APIリクエスト
                response = self.search_items(store_id, query, page)
                
                # エラーチェック
                if "error" in response:
                    logging.error(f"ストアID {store_id} のページ {page} での検索エラー")
                    continue
                
                # 検索統計情報を取得
                stats = {
                    'ストアID': store_id,
                    'ページ': page,
                    '総検索ヒット件数': response.get('totalResultsAvailable', 0),
                    '返却された商品件数': response.get('totalResultsReturned', 0),
                    '最初のデータ位置': response.get('firstResultPosition', 1),
                    '検索日時': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                }
                search_stats.append(stats)
                
                # 商品情報の取得
                hits = response.get('hits', [])
                if not hits:
                    logging.info(f"ストアID {store_id} のページ {page} で商品が見つかりませんでした")
                    break  # このストアの次のページはない
                
                # 各ヒットから商品データを抽出
                for hit in hits:
                    product_data = self.extract_product_data(hit)
                    all_products.append(product_data)
                
                logging.info(f"ストアID {store_id} のページ {page} から {len(hits)}件の商品情報を取得")
                
                # リクエスト間の待機時間（2秒）
                if page < max_pages:
                    logging.info("次のリクエストのために2秒待機します...")
                    time.sleep(2)
        
        # 商品データをデータフレームに変換
        products_df = pd.DataFrame(all_products)
        
        # 検索統計情報をデータフレームに変換
        stats_df = pd.DataFrame(search_stats)
        
        # CSVに保存
        if save_to_csv:
            # 商品情報の保存
            if not products_df.empty:
                csv_filename = self.config['yahoo_shopping']['output']['csv_filename']
                csv_path = os.path.join(self.data_dir, csv_filename)
                products_df.to_csv(csv_path, index=False, encoding='utf-8-sig')
                logging.info(f"{len(products_df)}件の商品情報を {csv_path} に保存しました")
                print(f"商品情報を {csv_path} に保存しました")
            
            # 検索統計情報の保存
            if not stats_df.empty:
                stats_csv_filename = 'yahoo_shopping_stats.csv'
                stats_csv_path = os.path.join(self.data_dir, stats_csv_filename)
                stats_df.to_csv(stats_csv_path, index=False, encoding='utf-8-sig')
                logging.info(f"{len(stats_df)}件の検索統計情報を {stats_csv_path} に保存しました")
                print(f"検索統計情報を {stats_csv_path} に保存しました")
        
        return products_df, stats_df


    def search_all_items_with_price_paging(self, store_id, query="", max_iterations=10, save_to_csv=True):
        """
        価格を基準にページングして、1000件以上の商品情報を取得する
        
        Parameters:
            store_id (str): ストアID
            query (str, optional): 検索キーワード
            max_iterations (int, optional): 最大繰り返し回数
            save_to_csv (bool, optional): CSVに保存するかどうか
            
        Returns:
            tuple: (商品情報のデータフレーム, 検索統計情報のデータフレーム)
        """
        # 全商品データの格納用リスト
        all_products = []
        
        # 検索統計情報の格納用リスト
        search_stats = []
        
        # 現在の最低価格（初回は指定なし）
        current_min_price = None
        
        # 最高価格の制限（10000円）
        max_price = 10000
        
        for iteration in range(1, max_iterations + 1):
            logging.info(f"=== イテレーション {iteration}/{max_iterations} ===")
            
            # 最大10ページ取得（合計1000件まで）
            iteration_products = []
            
            for page in range(1, 11):
                # APIリクエスト
                response = self.search_items(
                    store_id=store_id, 
                    query=query, 
                    page=page, 
                    sort="+price", 
                    price_from=current_min_price,
                    price_to=max_price
                )
                
                # エラーチェック
                if "error" in response:
                    logging.error(f"ストアID {store_id} のページ {page} での検索エラー")
                    continue
                
                # 検索統計情報を取得
                stats = {
                    'ストアID': store_id,
                    'イテレーション': iteration,
                    'ページ': page,
                    '最低価格': current_min_price,
                    '最高価格': max_price,
                    '総検索ヒット件数': response.get('totalResultsAvailable', 0),
                    '返却された商品件数': response.get('totalResultsReturned', 0),
                    '最初のデータ位置': response.get('firstResultPosition', 1),
                    '検索日時': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                }
                search_stats.append(stats)
                
                # 商品情報の取得
                hits = response.get('hits', [])
                if not hits:
                    logging.info(f"商品が見つかりませんでした")
                    break  # 商品がなければ終了
                
                # 各ヒットから商品データを抽出
                for hit in hits:
                    product_data = self.extract_product_data(hit)
                    iteration_products.append(product_data)
                
                logging.info(f"ページ {page} から {len(hits)}件の商品情報を取得 (合計: {len(iteration_products)}件)")
                
                # リクエスト間の待機時間（2秒）
                if page < 10 and hits:
                    logging.info("次のリクエストのために2秒待機します...")
                    time.sleep(2)
            
            # このイテレーションで商品が取得できなかった場合は終了
            if not iteration_products:
                logging.info(f"イテレーション {iteration} で商品が取得できませんでした。処理を終了します。")
                break
            
            # 取得した商品を全体リストに追加
            all_products.extend(iteration_products)
            
            # 次のイテレーションのための最低価格を設定
            # 取得した商品の最高価格+1を次の最低価格にする
            if iteration_products:
                # DataFrameに変換して最高価格を取得
                iteration_df = pd.DataFrame(iteration_products)
                if '価格' in iteration_df.columns and not iteration_df['価格'].empty:
                    iter_max_price = iteration_df['価格'].max()
                    current_min_price = iter_max_price + 1
                    logging.info(f"次のイテレーションの最低価格を {current_min_price}円 に設定")
                    
                    # 最高価格を超える場合は終了
                    if current_min_price >= max_price:
                        logging.info(f"最高価格 {max_price}円 に達したため、検索を終了します。")
                        break
                else:
                    logging.warning("商品価格が取得できませんでした。検索を終了します。")
                    break
            
            # 最大繰り返し回数に達した場合は終了
            if iteration >= max_iterations:
                logging.info(f"最大繰り返し回数 ({max_iterations}) に達しました。")
        
        # 商品データをデータフレームに変換
        products_df = pd.DataFrame(all_products)
        
        # 検索統計情報をデータフレームに変換
        stats_df = pd.DataFrame(search_stats)
        
        # CSVに保存
        if save_to_csv and not products_df.empty:
            # 設定ファイルから出力ファイル名を取得
            csv_filename = self.config['yahoo_shopping']['output']['csv_filename']
            csv_path = os.path.join(self.data_dir, csv_filename)
            products_df.to_csv(csv_path, index=False, encoding='utf-8-sig')
            logging.info(f"{len(products_df)}件の商品情報を {csv_path} に保存しました")
            print(f"商品情報を {csv_path} に保存しました")
            
            # 検索統計情報の保存
            stats_csv_filename = 'yahoo_shopping_stats.csv'
            stats_csv_path = os.path.join(self.data_dir, stats_csv_filename)
            stats_df.to_csv(stats_csv_path, index=False, encoding='utf-8-sig')
            logging.info(f"{len(stats_df)}件の検索統計情報を {stats_csv_path} に保存しました")
            print(f"検索統計情報を {stats_csv_path} に保存しました")
        
        return products_df, stats_df

In [ ]:
# # セル3: 基本的な検索のテスト（search_all_stores）
# if __name__ == "__main__":
#     # YahooShoppingAPIのインスタンスを作成
#     yahoo_api = YahooShoppingAPI()
    
#     # すべてのストアIDで商品検索を実行
#     products_df, stats_df = yahoo_api.search_all_stores(max_pages=10)
    
#     # 商品情報の表示
#     if not products_df.empty:
#         print("\n=== 検索結果のサンプル（最初の5件）===")
#         print(products_df.head(5))
#         print(f"\n合計 {len(products_df)} 件の商品情報を取得しました")
#     else:
#         print("商品情報が取得できませんでした")
    
#     # 検索統計情報の表示
#     if not stats_df.empty:
#         print("\n=== 検索統計情報 ===")
#         print(stats_df)
#         total_hits = stats_df['総検索ヒット件数'].sum()
#         print(f"\n全ストアの総検索ヒット件数: {total_hits}")

In [ ]:
# セル4: 価格ページング検索のテスト（search_all_items_with_price_paging）
if __name__ == "__main__":
    # YahooShoppingAPIのインスタンスを作成
    yahoo_api = YahooShoppingAPI()
    
    # 特定のストアIDで価格ページングを使って商品検索を実行
    store_id = "guruguru"  # 実際のストアIDに置き換える
    products_df, stats_df = yahoo_api.search_all_items_with_price_paging(store_id, max_iterations=3)
    
    # 商品情報の表示
    if not products_df.empty:
        print("\n=== 検索結果のサンプル（最初の5件）===")
        print(products_df.head(5))
        print(f"\n合計 {len(products_df)} 件の商品情報を取得しました")
        
        # 商品価格の分布
        if '価格' in products_df.columns:
            print("\n=== 価格分布 ===")
            price_stats = products_df['価格'].describe()
            print(price_stats)
            
            # 重複チェック
            if 'JAN' in products_df.columns:
                dupes = products_df[products_df.duplicated(['JAN'], keep=False)]
                if not dupes.empty:
                    print(f"\n重複するJANコードが {len(dupes)} 件あります")
    else:
        print("商品情報が取得できませんでした")
    
    # 検索統計情報の表示
    if not stats_df.empty:
        print("\n=== 検索統計情報のサマリー ===")
        print(f"総ページ数: {len(stats_df)}")
        print(f"総検索ヒット件数の最大値: {stats_df['総検索ヒット件数'].max()}")
        print(f"返却された商品件数の合計: {stats_df['返却された商品件数'].sum()}")